In [6]:
# 1. 필요 라이브러리 설치
# !pip install jupyter langchain-community langchain-pinecone pinecone-client sentence-transformers torch python-dotenv

# 2. 라이브러리 임포트
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
import os
from dotenv import load_dotenv

# --- API 키 설정 ---
load_dotenv()

# --- 데이터 및 벡터 스토어 준비 ---
try:
    chbook_info_df = pd.read_csv('chbook_info1358.csv')
    childrenbook_df = pd.read_csv('childrenbook.csv')
    merged_df = pd.merge(childrenbook_df, chbook_info_df, on='isbn13')
    merged_df['addition_symbol'] = merged_df['addition_symbol'].astype(str)
    filtered_df = merged_df[merged_df['addition_symbol'].str.startswith(('6', '7'))].copy()
    
    # 검색 정확도를 높이기 위해 책 제목, 설명, 키워드를 모두 사용합니다.
    filtered_df['text_for_embedding'] = filtered_df['bookname'] + " " + filtered_df['description'].fillna('').str.replace('\r\n', ' ', regex=False) + ' ' + filtered_df['keyword'].fillna('')
    filtered_df = filtered_df.fillna('')
    loader = DataFrameLoader(filtered_df, page_content_column='text_for_embedding')
    documents = loader.load()
    print("✅ 데이터 준비 완료!")
except FileNotFoundError as e:
    print("🚨 파일 로딩 오류: chbook_info.csv와 childrenbook.csv 파일이 .ipynb 파일과 같은 폴더에 있는지 확인해주세요.")
    exit()

model_name_for_embedding = "jhgan/ko-sroberta-nli"
hf_embeddings = HuggingFaceEmbeddings(model_name=model_name_for_embedding)
index_name = "children-books"

pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

if index_name not in pc.list_indexes().names():
    # print(f"Pinecone 인덱스 '{index_name}'를 새로 생성합니다...")
    pc.create_index(name=index_name, dimension=768, metric="cosine", spec=ServerlessSpec(cloud='aws', region='us-east-1'))
    # print(f"✅ 인덱스 생성 완료. 데이터 저장을 시작합니다...")
    PineconeVectorStore.from_documents(documents=documents, embedding=hf_embeddings, index_name=index_name)
    # print("✅ 데이터 저장 완료!")
else:
    # print(f"✅ 기존 인덱스 '{index_name}'를 사용합니다.")
    pass

vectorstore = PineconeVectorStore(index_name=index_name, embedding=hf_embeddings)
# ⭐️ LLM 없이 순수 검색만 수행할 '검색기'를 준비합니다.
retriever = vectorstore.as_retriever(search_kwargs={'k': 4}) # 검색할 책의 개수 (4개)
print(f"✅ 순수 검색 준비 완료!")


# --- 💬 순수 검색 실행 ---
print("\n✅ API 호출 없이 CSV 파일 내에서만 검색합니다.")
print("   (종료하려면 'exit' 입력)")

while True:
    try:
        user_input = input("\n👤 당신: ")
        if user_input.lower() == 'exit':
            print("🤖 검색기: 이용해주셔서 감사합니다!")
            break

        # retriever를 사용해 Pinecone에서 유사도가 높은 문서를 검색합니다.
        retrieved_docs = retriever.invoke(user_input)

        print(f"\n--- 📚 '{user_input}'에 대한 검색 결과 ---")
        if not retrieved_docs:
            print("검색된 책이 없습니다.")
        else:
            # ⭐️ 사용자님이 원하신 형식으로 출력합니다
            for i, doc in enumerate(retrieved_docs):
                # doc.metadata는 CSV의 한 행에 해당하는 모든 정보를 담고 있습니다.
                book_title = doc.metadata.get('bookname', '제목 없음')
                description = doc.metadata.get('description', '내용 없음')
                print(f"{i+1}. {book_title} : {description}")

    except Exception as e:
        print(f"🚨 검색 중 오류가 발생했습니다: {e}")
        break

✅ 데이터 준비 완료!
✅ 순수 검색 준비 완료!

✅ API 호출 없이 CSV 파일 내에서만 검색합니다.
   (종료하려면 'exit' 입력)

--- 📚 '전래동화 추천해줘'에 대한 검색 결과 ---
1. 똥벼락 : '똥과 도깨비'라는 전래동화의 소재를 절묘하게 조화시킨 그림 동화집. 옛날 옛그림을 연상시키는 그림체가 화면을 가득 채우고, 단순하면서도 옛이야기 특유의 과장을 살린 글솜씨가 돋보이는 그림책이다.
2. 돌이 척척 개구리 킁킁 : 수북수북 옛이야기 시리즈. 따스한 한지 인형과 구수한 입말이 어우러진 옛이야기 그림책이다. ‘돌이 척척 개구리 킁킁’ 이야기를 들려주신 분은 경기도 여주읍의 안평국 할아버지신데, 익살맞고 구수한 입말이 이야기의 재미를 한층 더해 주고 있다.
3. 지각대장 존 : 얼른 보면 재미만 노리는 동화처럼 보인다．그러나 한 두 번 반복해서 읽으면 중요한 의미가 암시되고 있는，하나의 중요한 교훈이 담긴 동화라는 것을 알게 된다．어린 아이의 사정이 어른의 눈에는 무의미할지 모르나 하늘의 눈에는 소중한 것이라는 사실을 가르치는 교훈이다.
4. 개와 고양이 : 옛이야기의 원형에 가장 가깝다는 평가를 받고 있는 &lt;조선전래동화집&gt;을 원전으로, 아이들의 눈높이에 맞추어 엮은 옛이야기 그림책. 동화 작가들이 박영만의 작가적 개성과 세계관 등이 맛깔스럽게 녹아 있는 원작의 표현과 말투를 잘 살리면서도 그림책을 읽는 어린이들에게 맞추어 솜씨 있게 다듬었다.

--- 📚 '공룡책 추천해줘'에 대한 검색 결과 ---
1. 배드 가이즈 : 전 세계 40개국 어린이들이 빠져든 화제의 시리즈&#x0D;the BAD GUYS 한국어판 출간!&#x0D;&#x0D;이걸 읽고 웃음 참을 수 있는 사람 있으면 나와 보시길.&#x0D;-[커커스 리뷰]&#x0D;&#x0D;그림책 다음 단계로 넘어가는 아이들, 책 읽기 좋아하고 웃기 좋아하는 아이들에게 추천한다. 잠들기 전 아이들과 함께 읽을 책을...
2. 오싹오싹 팬티! : 토토의 그림책 시리즈. 2013

In [5]:
print(merged_df.columns)

Index(['no', 'ranking', 'bookname', 'authors', 'publisher', 'publication_year',
       'isbn13', 'addition_symbol', 'vol', 'class_no', 'class_nm',
       'bookImageURL', 'bookDtlUrl', 'description', 'loanCnt', 'keyword'],
      dtype='object')
